In [8]:
from dotenv import load_dotenv
load_dotenv()
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
from dags.lib.PineconeManager import PineconeManager
import os
pinecone_key = os.getenv("PINECONE_API")
index_name = "letstalkvector"
namespace = "letstalk-ns"

pi = PineconeManager(index_name, namespace, pinecone_key)

In [40]:
interest = ""

response = pi.query(interest, num_results=5)

In [41]:
result = response.get('result').get('hits')
input_text = f"""Generate exactly ONE question about the following topic. Do NOT answer it. Make it FUN and INFORMAL.
Topic: {result}
Question:"""

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

for i, out in enumerate(outputs):
    full_text = tokenizer.decode(out, skip_special_tokens=True)
    question = full_text.split("Question:")[-1].split("\n")[0].split("?")[0].strip()
    print(f"Option {i+1}: {question}?")

Option 1: How many questions are there for each topic?
Option 2: What does "Fakiren fra Bilbao" refer to?
Option 3: What was Baia Arena's revenue?


In [36]:
response

{'result': {'hits': [{'_id': '73539',
                      '_score': 0.24178464710712433,
                      'fields': {'adult': False,
                                 'budget': 0.0,
                                 'genres': 'music',
                                 'original_language': 'en',
                                 'overview': 'baionarena is a live album and '
                                             'dvd release in 2009 by manu chao '
                                             'and radio bemba  it was recorded '
                                             'during summer 2008 in the arena '
                                             'of bayonne during the f tes de '
                                             'bayonne  hence the name '
                                             'baionarena which also happens to '
                                             'mean the one that belongs to '
                                             'bayonne in basque',
    